In [1]:
import re
from string import punctuation
import pandas as pd
from pandas import DataFrame
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import numpy as np

In [2]:
#Define where's the CSV file
filePath = 'C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/dataSetNewSteps.csv'

In [3]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath)

In [4]:
def fixDotsProblem(text):
    occurrences = re.findall('\.[A-Za-z]',text)
    for string in occurrences:
        fixing = string.replace('.','. ')
        text = text.replace(string,fixing)
    return text

In [5]:
def fixPunctuation(text):
    occurrences = re.findall('[áàãâäéèêëíìîïóòõôöúùûüA-Za-z][-./?!,":;()\'][áàãâäéèêëíìîïóòõôöúùûüA-Za-z]',text)
    for string in occurrences:
        fixing = re.sub('[-./?!,":;()\']+',' ', string)
        text = text.replace(string,fixing)
    return text

In [6]:
def removePunctuation(text):
    punctuationList = set(list(punctuation) + ['\u201c','\u201d','\u2013'])
    for signal in punctuationList:
        text = text.replace(signal,'')
    return text

In [7]:
def cleanExtraSpaces(text):
    text = re.sub('[ ]+',' ',text)
    return text

In [83]:
def removeExtraSpacesFromPunctuation(text):
    occurrences = re.findall('\S+ [.?!,:;)]+',text)
    for string in occurrences:
        fixing = re.sub(' ','', string)
        text = text.replace(string,fixing)
    return text

In [70]:
def cleanText(text):
    text = fixDotsProblem(text)
    text = fixPunctuation(text)
    #text = removePunctuation(text)
    text = cleanExtraSpaces(text)
    text = text.lower()
    return text

In [71]:
df['sinopse'] = [cleanText(str(synopsis)) for synopsis in df['sinopse']]

In [6]:
df = df.drop(['Unnamed: 0'],1)
print(df.shape)
print(df.dtypes)

(10712, 27)
isbn-10                object
isbn-13                 int64
titulo                 object
autor                  object
editora                object
ano                     int64
paginas                 int64
idioma                 object
leram                   int64
lendo                   int64
queremLer               int64
relendo                 int64
abandonos               int64
resenhas                int64
nota                  float64
favoritos               int64
desejados               int64
trocam                  int64
avaliaram               int64
cincoEstrelas         float64
quatroEstrelas        float64
tresEstrelas          float64
duasEstrelas          float64
umaEstrela            float64
avaliacoesHomens      float64
avaliacoesMulheres    float64
sinopse                object
dtype: object


In [7]:
df = df.dropna(axis=0,how='any',subset=['sinopse'],inplace=False)

In [8]:
def contarPalavrasNaSinopse():
    palavrasNaSinopse = []
    for sinopse in df['sinopse']:
        palavras = word_tokenize(sinopse)
        palavrasNaSinopse.append(len(palavras))
    print(len(palavrasNaSinopse))
    return palavrasNaSinopse

In [9]:
def extrairEstatisticas(palavrasNaSinopse):
    synopsisLength = np.array(palavrasNaSinopse)

    # Média
    print("Média: "+str(np.mean(synopsisLength)))

    # Mediana
    print("Mediana: "+str(np.median(synopsisLength)))

    # Máximo e Mínimo
    print("Máximo: "+str(np.max(synopsisLength)))
    print("Mínimo: "+str(np.min(synopsisLength)))

    # Intervalo médio
    print("Intervalo médio: "+str((np.max(synopsisLength)-np.min(synopsisLength))/2))

    # Variância e desvio-padrão
    print("Variância: "+str(np.var(synopsisLength,ddof=1)))
    print("desvio-padrão: "+str(np.std(synopsisLength,ddof=1)))

In [16]:
#list_stopwords = set(stopwords.words('portuguese'))
#def removeStopwords(text):
    #palavras = word_tokenize(text)
    #palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in list_stopwords]
    #final_string = " ".join(palavras_sem_stopwords)
    #final_string = removeExtraSpacesFromPunctuation(final_string)
    #return final_string

In [10]:
palavrasNaSinopse = contarPalavrasNaSinopse()
extrairEstatisticas(palavrasNaSinopse)
#df['sinopse']

10712
Média: 172.8859223300971
Mediana: 149.0
Máximo: 1536
Mínimo: 3
Intervalo médio: 766.5
Variância: 11044.402445811538
desvio-padrão: 105.09235198534448


In [15]:
#df['sinopse'] = [removeStopwords(sinopse) for sinopse in df['sinopse']]
#palavrasNaSinopse = contarPalavrasNaSinopse()
#extrairEstatisticas(palavrasNaSinopse)
#df['sinopse']

In [107]:
# I may need to run this command more than once
df['sinopse'] = [removeExtraSpacesFromPunctuation(text) for text in df['sinopse']]

In [37]:
def removeLinksFromSynopsis():
    df['sinopse'] = [re.sub('http\S+','',text) for text in df['sinopse']]
    df['sinopse'] = [re.sub('[^ ]www.\S+','',text) for text in df['sinopse']]
    df['sinopse'] = [re.sub('www.\S+','',text) for text in df['sinopse']]

In [97]:
removeLinksFromSynopsis()

In [161]:
def removePunctuationExcess(text):
    text = re.sub('[.]{4,}','...',text)
    text = re.sub('[!]{2,}','!',text)
    text = re.sub('[?]{2,}','?',text)
    text = re.sub('[=]{2,}','=',text)
    return text

In [168]:
#synopsis = df['sinopse']
for text in synopsis:
    occurrences = re.findall('["]{2,} \S+',text)
    if len(occurrences) > 0:
        print(text)

"o livro o nome de deus é misericórdia relata uma entrevista concedida ao vaticanista andrea tornielli, onde o papa francisco explica melhor o ano da misericórdia que teve inicio no dia 8 de dezembro de 215. a entrevista foi concedida em julho de 215, após a visita do papa a américa latina (equador, bolívia e paraguai). francisco recebeu o jornalista tornielli na casa santa marta, munido da bíblia e de citações dos padres da igreja. a misericórdia foi o tema da conversa, tendo em vista o jubileu extraordinário que seria aberto cinco meses depois. os frutos desse diálogo estão no livro lançado hoje. ""a igreja não está no mundo para condenar, mas para promover o encontro com aquele amor visceral que é a misericórdia de deus. para que isso aconteça, é necessário sair. sair das igrejas e das paróquias, sair e ir a procura das pessoas onde elas se encontram, onde sofrem, onde esperam."" neste livro o papa apresenta a sua visão sobre a missão da igreja no mundo, sublinhando que quando ""con

In [143]:
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [115]:
# I may need to run this command more than once
synopsis = [removePunctuationExcess(text) for text in synopsis]

In [108]:
#Export DataFrame to a new CSV file
df.to_csv(filePath)